# Allocation Algorithm 3

> **1.** Create child dataframe from MySQL locationdata table

> **2.** Create ei dataframe from MySQL table locationdata table

> **3.** Assign children to ei (children in random order).  After an EI has 17 children, no more children are assigned to them to create somewhat even distribution

> **4.** Push allocation to MySQL table locationdataalgo3

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("SELECT child_id, longitude, latitude FROM child ")
cursor.execute(query)

child_ids = cursor.fetchall()

data_list = []
for row in child_ids:
    data = {}
    data["id"] = row[0]
    data["longitude"] = row[1]
    data["latitude"] = row[2]
    child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_1 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child1_child_query)
    child1_child = cursor.fetchall()
    
    for row2 in child1_child:
        # Add columns with prefixes
        data[f'{row2[1]}_distance'] = row2[2]
        data[f'{row2[1]}_time'] = row2[3]
        
    child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_2 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_child2_query)
    child_child2 = cursor.fetchall()
    
    for row3 in child_child2:
        data[f'{row3[0]}_distance'] = row3[2]
        data[f'{row3[0]}_time'] = row3[3]
    
    child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE child_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_ei_query)
    child_ei = cursor.fetchall()
    
    for row4 in child_ei:
        data[f'{row4[1]}_distance'] = row4[2]
        data[f'{row4[1]}_time'] = row4[3]
        
    data_list.append(data)
        
childDataFrame = pd.DataFrame(data_list)
childDataFrame = childDataFrame.sample(frac=1)
childDataFrame.reset_index(drop=True, inplace=True)
childDataFrame["assigned"] = ""

print(childDataFrame)

                           id  longitude   latitude  \
0    65e5df8638f826f292672014 -82.336411  36.343914   
1    65d8c240e09e67e025e8cd91 -82.751457  36.556641   
2    65e5dbff38f826f29267200c -82.444618  36.412182   
3    65d8bcaae09e67e025e8cd87 -82.668076  36.578304   
4    65d8bb48e09e67e025e8cd85 -82.679001  36.544052   
..                        ...        ...        ...   
191  65d76430e09e67e025e8cd43 -82.178283  36.534504   
192  65d8ba3ae09e67e025e8cd83 -82.675079  36.548363   
193  65d77ed6e09e67e025e8cd60 -82.123184  36.583393   
194  65e5c8f938f826f292671fe9 -82.170753  36.286129   
195  65e5d9ad38f826f292672007 -81.751564  36.453678   

     65d8d732e09e67e025e8cdb6_distance  65d8d732e09e67e025e8cdb6_time  \
0                             24.55900                       36.98130   
1                             46.02130                       56.84690   
2                             26.35540                       37.38160   
3                             38.67650         

In [3]:
query = ("Select ei_id, longitude, latitude from ei ")
cursor.execute(query)

ei_ids = cursor.fetchall()

ei_list = []
for row in ei_ids:
    data = {}
    data["id"] = row[0]
    data["longitude"] = row[1]
    data["latitude"] = row[2]
    ei_child_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE ei_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(ei_child_query)
    ei_child = cursor.fetchall()
    
    for row2 in ei_child:
        # Add columns with prefixes
        data[f'{row2[0]}_distance'] = row2[2]
        data[f'{row2[0]}_time'] = row2[3]
        
    ei1_ei_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_1 = '{row[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei1_ei_query)
    ei1_ei = cursor.fetchall()
    
    for row3 in ei1_ei:
        data[f'{row3[1]}_distance'] = row3[2]
        data[f'{row3[1]}_time'] = row3[3]
    
    ei_ei2_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_2 = '{row[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei_ei2_query)
    ei_ei2 = cursor.fetchall()
    
    for row4 in ei_ei2:
        data[f'{row4[0]}_distance'] = row4[2]
        data[f'{row4[0]}_time'] = row4[3]
        
    ei_list.append(data)

eiDataFrame = pd.DataFrame(ei_list)
eiDataFrame = eiDataFrame[eiDataFrame['id'] != '65d75e28e09e67e025e8cd3c']  #drop this EI because she is currently not seeing children
eiDataFrame["assignment_count"] = 0
print(eiDataFrame)

                          id  longitude   latitude  \
0   65d75a2ce09e67e025e8cd30 -82.372108  36.319485   
1   65d75bb0e09e67e025e8cd31 -81.977836  36.722124   
2   65d75c59e09e67e025e8cd32 -82.944595  36.197052   
3   65d75c87e09e67e025e8cd33 -82.415306  36.305367   
4   65d75cb9e09e67e025e8cd34 -82.234230  36.391361   
5   65d75ce1e09e67e025e8cd35 -82.522095  36.509079   
6   65d75d08e09e67e025e8cd36 -82.490273  36.483723   
7   65d75d4ce09e67e025e8cd37 -82.460320  36.299648   
8   65d75d80e09e67e025e8cd38 -82.209000  36.592892   
9   65d75da9e09e67e025e8cd39 -82.354896  36.299501   
10  65d75dd5e09e67e025e8cd3a -82.383507  36.334129   
11  65d75dfae09e67e025e8cd3b -82.546295  36.537521   

    65d8d05fe09e67e025e8cda9_distance  65d8d05fe09e67e025e8cda9_time  \
0                             25.1360                        35.1757   
1                             74.1249                        82.4608   
2                             18.4015                        26.5191   
3        

In [4]:
#check to make sure both dataframes have all the neccessary columns
columns_only_in_child = set(childDataFrame.columns) - set(eiDataFrame.columns)
columns_only_in_ei = set(eiDataFrame.columns) - set(childDataFrame.columns)
print(columns_only_in_child)
print("/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
print(columns_only_in_ei)

{'assigned'}
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
{'assignment_count'}


In [5]:
#make sure both dataframes have the columns in the same order
childDataFrameOrdered = childDataFrame.sort_index(axis=1)
eiDataFrameOrdered = eiDataFrame.sort_index(axis=1)

print(childDataFrameOrdered)
print("///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
print(eiDataFrameOrdered)

     65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                              3.33131                        8.98799   
1                             35.05750                       45.13830   
2                              9.19936                       18.44090   
3                             29.03610                       37.47730   
4                             29.35300                       38.90990   
..                                 ...                            ...   
191                           23.07770                       34.04640   
192                           28.68040                       35.98380   
193                           26.11350                       39.20050   
194                           15.51820                       25.91930   
195                           48.49560                       72.59640   

     65d75bb0e09e67e025e8cd31_distance  65d75bb0e09e67e025e8cd31_time  \
0                              53.8236            

In [6]:
#the only null values in these dataframes should be from an entity to itself, which should always be 0
childDataFrameOrdered = childDataFrameOrdered.fillna(0)
eiDataFrameOrdered = eiDataFrameOrdered.fillna(0)

print(childDataFrameOrdered)
print("///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
print(eiDataFrameOrdered)

     65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                              3.33131                        8.98799   
1                             35.05750                       45.13830   
2                              9.19936                       18.44090   
3                             29.03610                       37.47730   
4                             29.35300                       38.90990   
..                                 ...                            ...   
191                           23.07770                       34.04640   
192                           28.68040                       35.98380   
193                           26.11350                       39.20050   
194                           15.51820                       25.91930   
195                           48.49560                       72.59640   

     65d75bb0e09e67e025e8cd31_distance  65d75bb0e09e67e025e8cd31_time  \
0                              53.8236            

In [7]:
def euclidean_distance(childRow, eiRow):
    child_values = childRow.drop(['id', 'assigned']).values
    ei_values = eiRow.drop(['id', 'assignment_count']).values
    return np.sqrt(np.sum((child_values - ei_values) ** 2))

In [8]:
childDataFrameAllocate = childDataFrameOrdered.copy()
for child_index, child_row in childDataFrameOrdered.iterrows():
    if childDataFrameAllocate.loc[child_index, "assigned"] == "":
        availableEIs = eiDataFrameOrdered[eiDataFrameOrdered['assignment_count'] < 17]
        eiDF = pd.DataFrame(columns=["id", "euclidean_distance"])
        for ei_index, ei_row in availableEIs.iterrows():
            e_value = euclidean_distance(child_row, ei_row)
            eiDF = pd.concat([eiDF, pd.DataFrame({"id": [ei_row["id"]], "euclidean_distance": [e_value]})], ignore_index=True)
        minValue = eiDF["euclidean_distance"].min()
        minIndex = eiDF["euclidean_distance"].idxmin()
        ei_with_min = eiDF.at[minIndex, "id"]

        same_location_children = childDataFrameOrdered[(childDataFrameOrdered["longitude"] == child_row["longitude"]) & 
                                                      (childDataFrameOrdered["latitude"] == child_row["latitude"])]
        childDataFrameOrdered.loc[same_location_children.index, 'assigned'] = ei_with_min
        
        rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == ei_with_min]
        count_ei_with_min = len(rows_with_ei_with_min)
        
        ei_index = eiDataFrameOrdered[eiDataFrameOrdered['id'] == ei_with_min].index
        eiDataFrameOrdered.at[ei_index[0], 'assignment_count'] = count_ei_with_min
    else:
        print("child already assigned")
    childDataFrameAllocate = childDataFrameOrdered.copy()

print(eiDataFrameOrdered)

child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
    65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                             0.00000                        0.00000   
1                            52.02660                       57.22680   
2                            36.75820                       48.24880   
3                             3.29009                        7.91787   
4                            13.81180                       23.98970   
5                            19.90670                       25.67660   
6                            17.1875

In [9]:
for index, ei in eiDataFrameOrdered.iterrows():
    ei_id = ei["id"]
    rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == ei_id]
    count_ei_with_min = len(rows_with_ei_with_min)
    print(f"children assigned to this ei: {count_ei_with_min}")

children assigned to this ei: 9
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17


In [10]:
print(childDataFrameOrdered)

     65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                              3.33131                        8.98799   
1                             35.05750                       45.13830   
2                              9.19936                       18.44090   
3                             29.03610                       37.47730   
4                             29.35300                       38.90990   
..                                 ...                            ...   
191                           23.07770                       34.04640   
192                           28.68040                       35.98380   
193                           26.11350                       39.20050   
194                           15.51820                       25.91930   
195                           48.49560                       72.59640   

     65d75bb0e09e67e025e8cd31_distance  65d75bb0e09e67e025e8cd31_time  \
0                              53.8236            

In [11]:
for index, child in childDataFrameOrdered.iterrows():
    child_id = child["id"]
    ei_id = child["assigned"]
    updateQuery = ("Update locationdataalgo3 "
                   "SET assigned = 1 "
                   f"WHERE child_id_1 = '{child_id}' and ei_id_1 = '{ei_id}' and child_id_2 is NULL and ei_id_2 is NULL")
    cursor.execute(updateQuery)
    connection.commit()
    print("Update query pushed")

Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed


In [12]:
cursor.close()
connection.close()